In [1]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \n    \'// setup cpp code highlighting\\n\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-cmake"] = {\\\'reg\\\':[/^%%cmake/]} ;\'\n)\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE, STDOUT, check_output\nimport html\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\nimport time\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    parser.add_argument("--under-spoiler-threshold", type=int, default=None)\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                comment_prefix = "%" + line_comment_start\n                if line.startswith(comment_prefix):\n                    cmds.append(\'#\' + line[len(comment_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        if cmd.startswith(\'#\'):\n            display(Markdown("\\#\\#\\#\\# `%s`" % cmd[1:]))\n        else:\n            display(Markdown("Run: `%s`" % cmd))\n            if args.under_spoiler_threshold:\n                out = check_output(cmd, stderr=STDOUT, shell=True, universal_newlines=True)\n                out = out[:-1] if out.endswith(\'\\n\') else out\n                out = html.escape(out)\n                if len(out.split(\'\\n\')) > args.under_spoiler_threshold:\n                    out = "<details> <summary> output </summary> <pre><code>%s</code></pre></details>" % out\n                elif out:\n                    out = "<pre><code>%s</code></pre>" % out\n                if out:\n                    display(HTML(out))\n            else:\n                get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef cmake(fname, cell):\n    save_file(fname, cell, "#")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    fname = fname or "makefile"\n    assert fname.endswith("makefile")\n    save_file(fname, cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    line = line.strip() \n    if line[0] == \'#\':\n        display(Markdown(line[1:].strip()))\n    else:\n        try:\n            expr, comment = line.split(" #")\n            display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n        except:\n            display(Markdown("{} = {}".format(line, eval(line))))\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def wait_stop(self, timeout):\n        for i in range(int(timeout * 10)):\n            wpid, status = os.waitpid(self.pid, os.WNOHANG)\n            if wpid != 0:\n                return True\n            time.sleep(0.1)\n        return False\n        \n    def close(self, timeout=3):\n        self.inq_f.close()\n        if not self.wait_stop(timeout):\n            os.kill(self.get_pid(), signal.SIGKILL)\n            os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Жизнь без стандартной библиотеки


<p><a href="https://www.youtube.com/watch?v=6_7ojZXErDU&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc&index=10" target="_blank">
    <h3>Видеозапись семинара</h3>
</a></p>

Что это значит? Значит, что функции взаимодействия с внешним миром (чтение, запись файлов и т. д.) будут реализованы в самом бинаре программы. Возможно, вы даже лично напишите их код.


[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/asm/nostdlib_baremetal) 


Сегодня в программе:
* <a href="#syscall" style="color:#856024"> Системные вызовы </a>
* <a href="#asm_32_64_diff" style="color:#856024"> Системные вызовы </a>
* <a href="#nolibc" style="color:#856024"> Пишем сами без libc </a>
* <a href="#brk" style="color:#856024"> Разбираемся в системным вызовом brk </a>
* <a href="#addr" style="color:#856024"> Развлекательная часть: смотрим на адреса различных переменных </a>
* <a href="#hw" style="color:#856024"> Комментарии к ДЗ </a>



# <a name="syscall"></a> Системные вызовы

In [253]:
%%cpp main.c
%run gcc -m64 main.c -o main.exe
%run strace ./main.exe 2> strace.out
%run cat strace.out

#include <stdio.h>

int main() {
    printf("Hello world!");
    return 0;
}

Run: `gcc -m64 main.c -o main.exe`

Run: `strace ./main.exe 2> strace.out`

Hello world!

Run: `cat strace.out`

execve("./main.exe", ["./main.exe"], 0x7fff454a30b0 /* 66 vars */) = 0
brk(NULL)                               = 0x561820b31000
arch_prctl(0x3001 /* ARCH_??? */, 0x7ffc4292e510) = -1 EINVAL (Invalid argument)
access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
openat(AT_FDCWD, "/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
fstat(3, {st_mode=S_IFREG|0644, st_size=79779, ...}) = 0
mmap(NULL, 79779, PROT_READ, MAP_PRIVATE, 3, 0) = 0x7ffacb0f1000
close(3)                                = 0
openat(AT_FDCWD, "/lib/x86_64-linux-gnu/libc.so.6", O_RDONLY|O_CLOEXEC) = 3
read(3, "\177ELF\2\1\1\3\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\0\360q\2\0\0\0\0\0"..., 832) = 832
pread64(3, "\6\0\0\0\4\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\0"..., 784, 64) = 784
pread64(3, "\4\0\0\0\20\0\0\0\5\0\0\0GNU\0\2\0\0\300\4\0\0\0\3\0\0\0\0\0\0\0", 32, 848) = 32
pread64(3, "\4\0\0\0\24\0\0\0\3\0\0\0GNU\0cBR\340\305\370\2609W\242\345)q\235A\1"..., 68, 880) = 68
fstat(3, {st_mode=S_IFREG|0755, 

## Компилим как обычно

In [256]:
%%cpp main.c --under-spoiler-threshold 5
%run gcc -m64 -masm=intel -fno-asynchronous-unwind-tables -O3 main.c -S -o main.S
%run gcc -m64 -masm=intel -O3 main.c -o main.exe
%run ls -la main.exe
%run ldd main.exe  # Выводим зависимости по динамическим библиотекам
%run cat main.S
%run objdump -M intel -d main.exe | grep main
%run strace ./main.exe

#include <unistd.h>

int main() {
    int w = write(1, "Hello world!", 12);
    return 0;
}

Run: `gcc -m64 -masm=intel -fno-asynchronous-unwind-tables -O3 main.c -S -o main.S`

Run: `gcc -m64 -masm=intel -O3 main.c -o main.exe`

Run: `ls -la main.exe`

Run: `ldd main.exe  # Выводим зависимости по динамическим библиотекам`

Run: `cat main.S`

Run: `objdump -M intel -d main.exe | grep main`

Run: `strace ./main.exe`

## Компилим, статически линкуя libc

In [257]:
%%cpp main2.c --under-spoiler-threshold 5
%run gcc -m64 -masm=intel -fno-asynchronous-unwind-tables -static -flto -O3 main2.c -S -o main2.S
%run gcc -m64 -masm=intel -static -flto -O3 main2.c -o main2.exe
%run ls -la main2.exe  # Заметьте, что размер стал сильно больше
%run ldd main2.exe || echo "fails with code=$?"
%run objdump -M intel -d main2.exe | grep "<_start>" -A 10 # Вот она функция с которой все начинается
%run objdump -M intel -d main2.exe | grep "<main>" -A 10 # Вот она функция main
%run objdump -M intel -d main2.exe | grep "<__libc_write>:" -A 8 | head -n 100 # А тут мы найдем syscall
%run strace ./main2.exe

#include <unistd.h>

int main() {
    int w = write(1, "X", 1);
    return 0;
}

Run: `gcc -m64 -masm=intel -fno-asynchronous-unwind-tables -static -flto -O3 main2.c -S -o main2.S`

Run: `gcc -m64 -masm=intel -static -flto -O3 main2.c -o main2.exe`

Run: `ls -la main2.exe  # Заметьте, что размер стал сильно больше`

Run: `ldd main2.exe || echo "fails with code=$?"`

Run: `objdump -M intel -d main2.exe | grep "<_start>" -A 10 # Вот она функция с которой все начинается`

Run: `objdump -M intel -d main2.exe | grep "<main>" -A 10 # Вот она функция main`

Run: `objdump -M intel -d main2.exe | grep "<__libc_write>:" -A 8 | head -n 100 # А тут мы найдем syscall`

Run: `strace ./main2.exe`

In [262]:
!gcc -E -m64 main2.c -o /dev/stdout | grep "main()" -A 10

int main() {
    int w = write(1, "X", 1);
    return 0;
}


Тут видим, что в `main` вызывается `__libc_write` (`write` либо макрос, либо соптимизировался), а в `__libc_write` происходит syscall с 0x1 в eax.

# <a name="asm_32_64_diff"></a> Отличие 32 и 64 битных архитектур в этом месте

Во первых номера системных вызовов разные

32 https://gist.github.com/yamnikov-oleg/454f48c3c45b735631f2

64 https://filippo.io/linux-syscall-table/

In [119]:
%%cpp main2.c --under-spoiler-threshold 20
%run gcc -m32 -masm=intel -static -flto -O3 main2.c -o main32.exe
%run gcc -m64 -masm=intel -static -flto -O3 main2.c -o main64.exe
%run objdump -M intel -d main64.exe | grep "<__libc_write>:" -A 8 | head -n 100 # Тут мы найдем syscall
%run objdump -M intel -d main32.exe | grep "<__libc_write>:" -A 15 | head -n 100 # А тут мы найдем...
%// call   DWORD PTR gs:0x10 -- это что-то про сегментные регситры и VDSO и там дальше сложно раскопать
%run objdump -M intel -d main32.exe | grep "<_exit>:" -A 9 | head -n 1000 # Попробуем покопать другую функцию - exit

#include <unistd.h>

int main() {
    int w = write(1, "X", 1);
    return 0;
}

Run: `gcc -m32 -masm=intel -static -flto -O3 main2.c -o main32.exe`

Run: `gcc -m64 -masm=intel -static -flto -O3 main2.c -o main64.exe`

Run: `objdump -M intel -d main64.exe | grep "<__libc_write>:" -A 8 | head -n 100 # Тут мы найдем syscall`

Run: `objdump -M intel -d main32.exe | grep "<__libc_write>:" -A 15 | head -n 100 # А тут мы найдем...`

\#\#\#\# `call   DWORD PTR gs:0x10 -- это что-то про сегментные регситры и VDSO и там дальше сложно раскопать`

Run: `objdump -M intel -d main32.exe | grep "<_exit>:" -A 9 | head -n 1000 # Попробуем покопать другую функцию - exit`

В 32-битной архитектуре системный вызов осуществляется с помощью `int 0x80`, в 64-битной - `syscall`.

Хотя в примере
```
mov    eax,0x1
int    0x80
```
это вызов exit на 32-битной архитектуре.

# <a name="nolibc"></a> Пишем сами без libc

In [132]:
%%cpp example1.c
%run gcc -m64 -masm=intel -nostdlib -O3 example1.c -o example1.exe
%run strace ./example1.exe || echo "Failed with code=$?" 

// Именно с этой функции всегда начинается выполнение программы
void _start() {
    
}

Run: `gcc -m64 -masm=intel -nostdlib -O3 example1.c -o example1.exe`

Run: `strace ./example1.exe || echo "Failed with code=$?"`

execve("./example1.exe", ["./example1.exe"], 0x7ffdec419330 /* 66 vars */) = 0
brk(NULL)                               = 0x557c5e16b000
arch_prctl(0x3001 /* ARCH_??? */, 0x7ffe540c0000) = -1 EINVAL (Invalid argument)
access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
mmap(NULL, 8192, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0x7f8dfbb50000
arch_prctl(ARCH_SET_FS, 0x7f8dfbb50a80) = 0
mprotect(0x557c5dbc8000, 4096, PROT_READ) = 0
--- SIGSEGV {si_signo=SIGSEGV, si_code=SEGV_MAPERR, si_addr=0x1} ---
+++ killed by SIGSEGV (core dumped) +++
Segmentation fault (core dumped)
Failed with code=139


Получаем сегфолт, так эта функция - входная точка в программу и в нее не передается разумного адреса возврата.
То есть делать return из этой функции - путь в никуда.

Интересный факт, что до старта `_start` все-таки есть жизнь, насколько я понимаю, это работа загрузчика программы и выполнение функций из секции `preinitarray`.

In [133]:
%%cpp exit.c
%run gcc -m64 -masm=intel -O3 exit.c -o exit.exe
%run strace ./exit.exe 2>&1 | tail -n 3

// обычная программа с пустым main
int main() {}

Run: `gcc -m64 -masm=intel -O3 exit.c -o exit.exe`

Run: `strace ./exit.exe 2>&1 | tail -n 3`

munmap(0x7efe42a4a000, 79779)           = 0
exit_group(0)                           = ?
+++ exited with 0 +++


Выполнение обычной программы заканчивается системным вызовом exit_group как подсказывает нам strace.

Можно нагуглить про этот системный вызов, например, это https://filippo.io/linux-syscall-table/

In [270]:
%%cpp example2.c
%run gcc -m64 -masm=intel -nostdlib -O3 example2.c -o example2.exe
%run strace ./example2.exe ; echo "Exited with code=$?" 
    
int my_exit(int code);
__asm__(R"(
my_exit:
    mov rax, 231 /* номер системного вызова exit_group */
    syscall
    /* не нужно возвращаться из функции, на этом программа завершится */
)");
    
// Именно с этой функции всегда начинается выполнение программы
void _start() {
    my_exit(0);
}

Run: `gcc -m64 -masm=intel -nostdlib -O3 example2.c -o example2.exe`

Run: `strace ./example2.exe ; echo "Exited with code=$?"`

execve("./example2.exe", ["./example2.exe"], 0x7ffc21ca4cf0 /* 66 vars */) = 0
brk(NULL)                               = 0x559965779000
arch_prctl(0x3001 /* ARCH_??? */, 0x7ffd3ff49220) = -1 EINVAL (Invalid argument)
access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
mmap(NULL, 8192, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0x7f5522d57000
arch_prctl(ARCH_SET_FS, 0x7f5522d57a80) = 0
mprotect(0x5599639c7000, 4096, PROT_READ) = 0
exit_group(0)                           = ?
+++ exited with 0 +++
Exited with code=0


Научились завершать нашу программу, отлично :)

Но не писать же всегда странные числа вроде 231?

`sudo apt-get install libc6-dev-amd64` - надо установить, чтобы `sys/syscall.h` нашелся.

In [154]:
%%cpp example3.c
%run gcc -m64 -masm=intel -nostdlib -O3 example3.c -o example3.exe
%run ./example3.exe ; echo "Exited with code=$?" 

// Тут есть макросы с номерами системных вызовов
#include <sys/syscall.h>
    
// Превращение макроса с числовым литералом в строковый литерал
#define stringify_impl(x) #x
#define stringify(x) stringify_impl(x)

    
int my_exit(int code);
__asm__(R"(
my_exit:
    mov rax, )" stringify(SYS_exit_group) R"( /* В разрыв строкового литерала ассемблерной вставки вставляется строковый литерал системного вызова */
    syscall
    /* не нужно возвращаться из функции, на этом программа завершится */
)");
    
// Именно с этой функции всегда начинается выполнение программы
void _start() {
    my_exit(0);
}

Run: `gcc -m64 -masm=intel -nostdlib -O3 example3.c -o example3.exe`

Run: `./example3.exe ; echo "Exited with code=$?"`

Exited with code=0


Можно так вывернуться, например.

Да, все что я пишу на Си, можно писать и просто на ассемблере. И инклюды там так же работают :)

In [271]:
%%asm example4.S
%//run gcc -E -m64 -masm=intel -nostdlib -O3 example4.S -o /dev/stdout
%run gcc -m64 -masm=intel -nostdlib -O3 example4.S -o example4.exe
%run ./example4.exe ; echo "Exited with code=$?" 

#include <sys/syscall.h>

.intel_syntax noprefix
.text

my_exit:
    mov rax, SYS_exit_group
    syscall
 
.globl _start
_start:
    mov rdi, 0
    call my_exit

\#\#\#\# `run gcc -E -m64 -masm=intel -nostdlib -O3 example4.S -o /dev/stdout`

Run: `gcc -m64 -masm=intel -nostdlib -O3 example4.S -o example4.exe`

Run: `./example4.exe ; echo "Exited with code=$?"`

Exited with code=0


Давайте напишем что-нибудь более сложное:


https://stackoverflow.com/questions/2535989/what-are-the-calling-conventions-for-unix-linux-system-calls-and-user-space-f

In [275]:
%%cpp minimal.c
%run gcc -g -m64 -masm=intel -nostdlib -O3 minimal.c -o minimal.exe
%run gcc -m64 -masm=intel -nostdlib -fno-asynchronous-unwind-tables -O3 minimal.c -S -o minimal.S
%run ls -la minimal.exe  # Заметьте, что размер стал очень маленьким :)
//%run ldd minimal.exe

//%run cat minimal.S
//%run objdump -d minimal.exe

%run ./minimal.exe ; echo "Exit code = $?" 

#include <sys/syscall.h>

    
// Универсальная функция для совершения системных вызовов (до 5 аргументов системного вызова)
int syscall(int code, ...);
__asm__(R"(
syscall:
    /* Function arguments: rdi, rsi, rdx, rcx, r8, r9 */
    /* Syscall arguments: rax (syscall num), rdi, rsi, rdx, r10, r8, r9.*/
    mov rax, rdi 
    mov rdi, rsi 
    mov rsi, rdx
    mov rdx, rcx
    mov r10, r8
    mov r8, r9
    syscall
    ret
)");

void my_exit(int code) {
    syscall(SYS_exit, code);
}

int write(int fd, const void* data, int size) {
    return syscall(SYS_write, fd, data, size);
}


void int_to_s(unsigned int i, char* s, int* len) {
    int clen = 0;
    for (int ic = i; ic; ic /= 10, ++clen);
    clen = clen ?: 1;
    s[clen] = '\0';
    for (int j = 0; j < clen; ++j, i /= 10) {
        s[clen - j - 1] = '0' + i % 10;
    }
    *len = clen;
}

unsigned int s_to_int(char* s) {
    unsigned int res = 0;
    while ('0' <= *s && *s <= '9') {
        res *= 10;
        res += *s - '0';
        ++s;
    }
    return res;
}

int print_int(int fd, unsigned int i) {
    char s[20];
    int len;
    int_to_s(i, s, &len);
    return write(fd, s, len);
}

int print_s(int fd, const char* s) {
    int len = 0;
    while (s[len]) ++len;
    return write(fd, s, len);
}




const char hello_s[] = "Hello world from function 'write'!\n";
const int hello_s_size = sizeof(hello_s);

// Забавно, но перед вызовом функции start стек не был выровнен по 16 :)
// Вернее был, но видимо не положили адрес возврата (так как не нужен), а сишный компилятор его ожидает...
__asm__(R"(
.globl _start
_start:
    sub rsp, 8
    jmp main
)");


void main() {
    const char hello_s_2[] = "Hello world from 'syscall'!\n";
    write(1, hello_s, sizeof(hello_s) - 1);
    syscall(SYS_write, 1, hello_s_2, sizeof(hello_s_2) - 1);
    print_s(1, "Look at this value: "); print_int(1, 10050042); print_s(1, "\n");
    print_s(1, "Look at this value: "); print_int(1, s_to_int("123456")); print_s(1, "\n");
    
    my_exit(0);
}


Run: `gcc -g -m64 -masm=intel -nostdlib -O3 minimal.c -o minimal.exe`

Run: `gcc -m64 -masm=intel -nostdlib -fno-asynchronous-unwind-tables -O3 minimal.c -S -o minimal.S`

Run: `ls -la minimal.exe  # Заметьте, что размер стал очень маленьким :)`

-rwxrwxr-x 1 pechatnov pechatnov 22400 окт 29 18:03 minimal.exe


Run: `./minimal.exe ; echo "Exit code = $?"`

Hello world from function 'write'!
Hello world from 'syscall'!
Look at this value: 10050042
Look at this value: 123456
Exit code = 0


# <a name="brk"></a> Разбираемся в системным вызовом brk

`void *sbrk(intptr_t increment);`

In [7]:
%%cpp minimal.c
%run gcc -m64 -masm=intel -nostdlib -O3 minimal.c -o minimal.exe
%run gcc -m64 -masm=intel -nostdlib -fno-asynchronous-unwind-tables -O3 minimal.c -S -o minimal.S

//%run cat minimal.S
//%run objdump -d minimal.exe

%run ./minimal.exe ; echo $? 

#include <sys/syscall.h>
#include <stdint.h>

    
// Универсальная функция для совершения системных вызовов (до 5 аргументов системного вызова)
int64_t syscall(int64_t code, ...);
__asm__(R"(
syscall:
    /* Function arguments: rdi, rsi, rdx, rcx, r8, r9 */
    /* Syscall arguments: rax (syscall num), rdi, rsi, rdx, r10, r8, r9.*/
    mov rax, rdi 
    mov rdi, rsi 
    mov rsi, rdx
    mov rdx, rcx
    mov r10, r8
    mov r8, r9
    syscall
    ret
)");

void my_exit(int code) {
    syscall(SYS_exit, code);
}

int64_t write(int fd, const void* data, int64_t size) {
    return syscall(SYS_write, fd, data, size);
}

void int_to_s(uint64_t i, char* s, int* len) {
    int clen = 0;
    for (int ic = i; ic; ic /= 10, ++clen);
    clen = clen ?: 1;
    s[clen] = '\0';
    for (int j = 0; j < clen; ++j, i /= 10) {
        s[clen - j - 1] = '0' + i % 10;
    }
    *len = clen;
}

unsigned int s_to_int(char* s) {
    unsigned int res = 0;
    while ('0' <= *s && *s <= '9') {
        res *= 10;
        res += *s - '0';
        ++s;
    }
    return res;
}

int print_int(int fd, int64_t i) {
    char s[40];
    int len;
    int_to_s(i, s, &len);
    return syscall(SYS_write, fd, s, len);
}

int print_s(int fd, const char* s) {
    int len = 0;
    while (s[len]) ++len;
    return syscall(SYS_write, fd, s, len);
}


const char hello_s[] = "Hello world from function 'write'!\n";
const int hello_s_size = sizeof(hello_s);


// Именно с этой функции всегда начинается выполнение программы
void _start() {
    const int size = 100 * 1000 * 1000;
    int* data_start = (void*)syscall(SYS_brk, 0);
    int* data_end = (void*)syscall(SYS_brk, (int*)data_start + size);
    
    print_s(1, "Data begin: "); print_int(1, (int64_t)(void*)data_start); print_s(1, "\n");
    print_s(1, "Data end: ");  print_int(1, (int64_t)(void*)data_end); print_s(1, "\n");
    
    data_start[0] = 1;
    for (int i = 1; i < (data_end - data_start); ++i) {
        data_start[i] = data_start[i - 1] + 1;
    }
    
    print_int(1, data_end[-1]); print_s(1, "\n");
    
    my_exit(0);
}


Run: `gcc -m64 -masm=intel -nostdlib -O3 minimal.c -o minimal.exe`

Run: `gcc -m64 -masm=intel -nostdlib -fno-asynchronous-unwind-tables -O3 minimal.c -S -o minimal.S`

Run: `./minimal.exe ; echo $?`

Data begin: 7261493248
Data end: 7661493248
100000000
0


# <a name="addr"></a> Смотрим на адреса различных переменных. Проверяем, что секции памяти расположены так, как мы ожидаем

In [4]:
%%cpp look_at_addresses.c --under-spoiler-threshold 30
%run gcc -m64 -masm=intel -O0 look_at_addresses.c -o look_at_addresses.exe
%run ./look_at_addresses.exe
%run gcc -S -m64 -masm=intel -Os look_at_addresses.c -o /dev/stdout

#include <stdio.h>
#include <stdlib.h>

int func(int a) {
    return a;
}


int* func_static_initialized() {
    static int a = 4;
    return &a;
}

const int* func_static_const_initialized() {
    static const int a = 4;
    return &a;
}

int* func_static_not_initialized() {
    static int a;
    return &a;
}


int global_initialized[3] = {1, 2, 3};
const int global_const_initialized[3] = {1, 2, 3};
int global_not_initialized[3];

int main2() {
   int local2 = 5;
   printf("Local 'local2' addr = %p\n", &local2); 
}


int main() {
    int local = 1;
    static int st = 2;
    int* all = malloc(12);
    
    printf("Func func addr = %p\n", (void*)func);
    printf("Func func_s addr = %p\n", (void*)func_static_initialized);
    
    printf("Global var (initialized) addr = %p\n", global_initialized);
    printf("Global var (const initialized) addr = %p\n", global_const_initialized);
    printf("Global var (not initialized) addr = %p\n", global_not_initialized);
    
    printf("Static 'st' addr = %p\n", &st);
    printf("Static 'func_static_initialized.a' addr = %p\n", func_static_initialized());
    printf("Static 'func_static_const_initialized.a' addr = %p\n", func_static_const_initialized());
    printf("Static 'func_static_not_initialized.a' addr = %p\n", func_static_not_initialized());
    
    printf("Local 'local' addr = %p\n", &local);
    main2();
    
    printf("Heap 'all' addr = %p\n", all);
    free(all);
    return 0;
}

Run: `gcc -m64 -masm=intel -O0 look_at_addresses.c -o look_at_addresses.exe`

Run: `./look_at_addresses.exe`

Run: `gcc -S -m64 -masm=intel -Os look_at_addresses.c -o /dev/stdout`

# <a name="syscall"></a> Комментарии к ДЗ
* Делайте выравнивание стека перед первой сишной функцией, если реализуете точку входа _start! Иначе, если оптимизатор заиспользует SSE у вас все разнесет.
* На сервере есть sys/syscall.h, как установить его локально в ноутбуке написано. Используйте константы оттуда! 
* Не надо мешать 231 и SYS_WRITE, это плохо
* Синтаксис для ассемблерных вставок по умолчанию AT&T. Но вы можете в начале вставки переключить его на intel, а потом обратно на AT&T. Пример, как это сделать, есть в первом семинаре по ассемблеру x86.